In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from numpy import linalg as la
import sys
import matplotlib.pyplot as plt
import seaborn as sas
import os
import csv
import re

In [2]:
# dict Position, file
label_Exp1 = {
    '1': 'Supine',
    '2': 'Right',
    '3': 'Left',
    '4': 'Right',
    '5': 'Right',
    '6': 'Left',
    '7': 'Left',
    '8': 'Supine',
    '9': 'Supine',
    '10': 'Supine',
    '11': 'Supine',
    '12': 'Supine',
    '13': 'Right',
    '14': 'Left',
    '15': 'Supine',
    '16': 'Supine',
    '17': 'Supine'
}

label_Exp2 = {
    'B1': 'Supine',
    'B2': 'Supine',
    'B3': 'Supine',
    'B4': 'Supine',
    'B5': 'Supine',
    'B6': 'Supine',
    'B7': 'Supine',
    'B8': 'Supine',
    'B9': 'Supine',
    'B10': 'Supine',
    'C1': 'Right',
    'C2': 'Right',
    'C3': 'Right',
    'D1': 'Left',
    'D2': 'Left',
    'D3': 'Left',
    'E1': 'Right',
    'E2': 'Right',
    'E3': 'Left',
    'E4': 'Left',
    'E5': 'Right',
    'E6': 'Left',
    'F1': 'Supine',
    'F2': 'Supine',
    'F3': 'Supine',
    'F4': 'Supine',
    'F5': 'Supine',
    'F6': 'Supine',
    'F7': 'Supine'
}


In [3]:
class Dataset:
    def __init__(self, values, subject, position):
        self.values = values
        self.subject = subject
        self.position = position
        

In [4]:
def PCA(file):
    df = pd.read_csv(file, sep="\t", header=None)
    df.drop(columns=[2048], inplace = True) 
    df.drop(0, inplace = True) 
    df.drop(1, inplace = True)
    df.drop(2, inplace = True)
    
    array = df.to_numpy()
    avg = np.mean(array, axis = 1, keepdims=True)

    array = array - avg

    CX = np.cov(array)
    w,v = la.eig(CX) 

    CY =  np.diag(np.round(np.real(w),2))

    P = v[:,0:10]
    P = abs(P)

    array_final = P.T.dot(array)
        
    return array_final
    
def padding(old_map):
    start = np.zeros((64, 3))
    final = np.zeros((64, 2))
    new_map = np.hstack((start, old_map, final))
    
    return new_map

def flip(pressure_map, experiment):
    if experiment == 1:
        pressure_map = np.flip(pressure_map, 1)
    else:
        pressure_map = np.flip(pressure_map, 0)
    
    return pressure_map
    
def trueLabel(num_file, experiment):
    if experiment == 1:
        position = label_Exp1.get(num_file)
    else: 
        position = label_Exp2.get(num_file)
        
    return position

def clean_string(string, experiment): 
    if experiment == 1:
        file = string[24:27]
        file = re.sub('[./_t]', '', file)
    
    else:
        if 'Air' in string:
            file = string[44:48]
            file = re.sub('[./_t]', '', file)
        else:
            file = string[50:54]
            file = re.sub('[./_t]', '', file)

    return file

def label_position(position):   
    if position == 'Supine':
        return 0
    
    if position == 'Right':
        return 1
    
    if position == 'Left':
        return 2

def rescale(pressure_map, max_value, min_value):  
    maximum = pressure_map.max()
    minimum = pressure_map.min()  
    OldRange = maximum - minimum
    NewRange = max_value - min_value

    for i in range(0,len(pressure_map)):
        pressure_map[i] = (((pressure_map[i] - minimum) * NewRange) / OldRange) 
            
    return pressure_map


In [5]:
# collects data from experiment 1
rootdir = Path("dataset/experiment-i/")
exp = 1

pressure_map = []
dataset = []

for subdir, dirs, files in os.walk(rootdir):
    for file in files:

        file_name = os.path.join(subdir, file)
        
        pressure = PCA(file_name)
           
        for i in range(0, len(pressure)):
            
            subject = file_name[22:24]
            if subject[1:2] == '/':
                subject = subject[0]

            file = clean_string(file_name, exp)
            position = trueLabel(file, exp)
            position = label_position(position)
        
            pressure_map = flip(pressure[i].reshape((64, 32)), exp)
            pressure_map = rescale(pressure_map, 500, 0)
            pressure_map = pressure_map.reshape((2048, ))
            
            data = Dataset(pressure_map, int(subject)-1, int(position))
            dataset.append(data)
        

In [6]:
# collects data from experiment 2
rootdir = Path("dataset/experiment-ii/")
exp = 2

pressure_map = []

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        
        file_name = os.path.join(subdir, file)
        
        #TODO: remove this if
        if '.DS_Store' in file_name:
            continue
            
        subject = file_name[23:25]
        if subject[1:2] == '/':
                subject = subject[0]
        
        file = clean_string(file_name, exp)
        position = trueLabel(file, exp)
        position = label_position(position)
        
        df = pd.read_csv(file_name, sep=" ", header=None)
        pressure_map = df.to_numpy()
        
        pressure_map = flip(pressure_map, exp)
        pressure_map = padding(pressure_map)
        pressure_map = pressure_map.reshape((2048, ))
        pressure_map = rescale(pressure_map, 500, 0)

                
        data = Dataset(pressure_map, int(subject)-1, int(position))
        dataset.append(data)


In [7]:
# write in csv
headers = []

for i in range(1, 65):
    for j in range(1, 33):
        header = "{} x {}".format(i, j)
        headers.append(header)

headers.append('Subject')
headers.append('Position')

with open('dataset.csv', mode='w') as csv_file:
    fieldnames = headers
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    
    for item in dataset:
        row = {}
        
        for i in range(0, len(item.values) + 2):
            
            if headers[i] == 'Subject':
                row[headers[i]] = item.subject

            elif headers[i] == 'Position':
                row[headers[i]] = item.position

            else:
                row[headers[i]] = item.values[i]
        
        writer.writerow(row)
